In [1]:
library(RPostgreSQL)
library(tidyverse)

Loading required package: DBI
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
files <- c("vitals", "labs", "blood-gas",
           "blood-gas-arterial", "gcs",
           "urine-output", "sofa") %>% paste0(".sql")
files

[1] "vitals.sql"             "labs.sql"               "blood-gas.sql"         
[4] "blood-gas-arterial.sql" "gcs.sql"                "urine-output.sql"      
[7] "sofa.sql"

In [6]:
c("second_day", "third_day") %>%
walk(function(d) {
    files %>% walk(function(f) {
        cat(sprintf("-- Run %s from %s --\n", f, d))
        
        f %>%
        file.path(sql_dir, d, .) %>%
        readr::read_file() %>%
        dbSendQuery(con, .)
    })
})

-- Run vitals.sql from second_day --
-- Run labs.sql from second_day --
-- Run blood-gas.sql from second_day --
-- Run blood-gas-arterial.sql from second_day --
-- Run gcs.sql from second_day --
-- Run urine-output.sql from second_day --
-- Run sofa.sql from second_day --
-- Run vitals.sql from third_day --
-- Run labs.sql from third_day --
-- Run blood-gas.sql from third_day --
-- Run blood-gas-arterial.sql from third_day --
-- Run gcs.sql from third_day --
-- Run urine-output.sql from third_day --
-- Run sofa.sql from third_day --


In [9]:
cat("-- Generate fluid data --\n")

c("fluid_input", "fluid_output", "fluid") %>%
walk(function(f) {
    f %>%
    paste0(".sql") %>%
    file.path(sql_dir, .) %>%
    file_to_sql_view(f) %>%
    dbSendQuery(con, .)
})

-- Generate fluid data --


In [12]:
cat("-- Generate lactate/creatinine reduction data --\n")

c("serum_non_diff", "serum_echo_diff", "serum_diff") %>%
walk(function(f) {
    f %>%
    paste0(".sql") %>%
    file.path(sql_dir, .) %>%
    file_to_sql_view(f) %>%
    dbSendQuery(con, .)
})

-- Generate lactate/creatinine reduction data --


In [13]:
cat("-- Generate view subgroup --\n")

sql <- file_to_sql_view(file.path(sql_dir, "subgroup.sql"),
                        "subgroup")
dbSendQuery(con, sql)

-- Generate view subgroup --


<PostgreSQLResult>

In [14]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE